<a href="https://colab.research.google.com/github/ethanmjansen/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS10_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

#Train/Test Split

In [288]:
#Looking at what feature contains times
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,perks,total_room,bed_bath
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,2.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3.0,2.0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2.0,1.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2.0,1.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5.0,4.0


In [0]:
#Changing created object to datetime 
df['created'] = pd.to_datetime(df['created'])

In [0]:
#Creating train and test
train = df[(df['created'] >= '2016-04') & (df['created'] <= '2016-05')]
test = df[df['created'] > '2016-05']

In [319]:
#Checking to make sure they have the same number of features
train.shape, test.shape

((16217, 37), (32600, 37))

#Engineering Two New Features

In [0]:
#First new feature total number of perks
df['perks']= df.iloc[:, 10:34].sum(axis=1)

In [293]:
#Checking feature for null values
df['perks'].isnull().sum()

0

In [0]:
#Second new feature total rooms
df['total_room'] = df['bedrooms'] + df['bathrooms']

In [295]:
#Checking feature for null values
df['total_room'].isnull().sum()

0

In [0]:
#Third New feature
df['bed_bath'] = df['bedrooms'] / df['bathrooms']

In [0]:
df['bed_bath'] = df['bed_bath'].replace(np.inf, np.nan)

In [0]:
df['bed_bath'] = df['bed_bath'].fillna(value=0)

In [326]:
df['bed_bath'].isnull().sum()

0

In [322]:
test['bed_bath'].describe()

count    32600.000000
mean         1.269172
std          0.877696
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          6.000000
Name: bed_bath, dtype: float64

#Fit Linear Regression Model With New Features

In [0]:
#Import Estimator Class
from sklearn.linear_model import LinearRegression

In [0]:
#Instantiate Class
model = LinearRegression()

In [0]:
#X matrix and y vector
features = ['perks', 'total_room', 'latitude', 'longitude', 'bed_bath']
target = 'price'
#X_train and X_test
X_train = train[features]
X_test = test[features]
#y_train and y_test
y_train = train[target]
y_test = test[target]

In [330]:
#Fit model to trains
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [331]:
#Fit model to tests
model.fit(X_test, y_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

#Model Coefficient and Intercept

In [332]:
Coefficient = model.coef_
Intercept = model.intercept_

print(f"The Model's Coefficient's are {Coefficient} and the Intercept is {Intercept}")

The Model's Coefficient's are [    49.54175599   1103.02516834   1916.03091591 -15130.8830255
   -711.23553978] and the Intercept is -1196116.791880902


#Regression Metrics

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [334]:
# Train Metrics

y_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
print(f'My MAE in my train data is ${mae:.2f}')
print(f'My RSME in my train data is ${rmse:.2f}')
print(f'My R^2 in my train data is {r2:}')

My MAE in my train data is $739.98
My RSME in my train data is $1147.21
My R^2 in my train data is 0.5588556146438686


In [335]:
# Test Metrics

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'My MAE in my train data is ${mae:.2f}')
print(f'My RSME in my train data is ${rmse:.2f}')
print(f'My R^2 in my train data is {r2:}')

My MAE in my train data is $741.44
My RSME in my train data is $1137.42
My R^2 in my train data is 0.591411511111233
